In [54]:
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
# List of top 25 subreddit names to read files
subredditNames = ["AmItheAsshole", "AskReddit", "Damnthatsinteresting", "DestinyTheGame", 
                  "Home", "LivestreamFail", "NoStupidQuestions", "PublicFreakout", "Unexpected", 
                  "WhitePeopleTwitter", "antiwork", "diablo4", "explainlikeimfive", "facepalm", 
                  "funny", "gaming", "interestingasfuck", "leagueoflegends", "mildlyinfuriating", 
                  "movies", "pcmasterrace", "pics", "therewasanattempt", "videos", "worldnews"]

In [31]:
# Updates the true count of tokens in subreddit
def UpdateTrueCount(trueCount, token):
    if token in trueCount:
        trueCount[token] += 1
    else:
        trueCount[token] = 1

In [35]:
def FalsePositiveRate(buffer, trueCount, m, k):
    fpr = 0
    for key in buffer.keys():
        if trueCount[key] < (m / k):
            fpr += 1
        
    return fpr / len(buffer)

In [3]:
# Basic Misra-Gries algorithm
def MisraGries(token, buffer, k): 
    if token in buffer:
        buffer[token] = buffer[token] + 1
    elif len(buffer) < k-1:
        buffer[token] = 1
    else:
        keys = list(buffer.keys())
        for key in keys:
            buffer[key] -= 1
            if buffer[key] == 0:
                del buffer[key]

In [39]:
# Needs uncommenting if not downloaded
# nltk.download('omw-1.4')
# nltk.download('stopwords')
# nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')
stop.append("like")
stop.append("get")

# Checks if a word is a common word, url, and lemmatizes the word
def CommonWord(word):
    isCommonWord = False

    if len(word) <= 1 or word in stop: isCommonWord = True
    elif word.startswith('https://') or word.startswith('http://'): isCommonWord = True

    # Performs Lemmatization
    word = wordnet_lemmatizer.lemmatize(word)
    
    return word, isCommonWord

In [55]:
for subreddit in subredditNames:
    print(subreddit)
    with open('Reddit_Comments/' + subreddit + '.txt') as f:
        buffer = {}
        trueCount = {}
        k = 801
        m = 0
        for line in f:
            tokens = line.split()
            random.shuffle(tokens) # Randomizes the tokens
            for token in tokens:
                token, isCommonWord = CommonWord(token)
                if isCommonWord: continue
                
                m += 1
                MisraGries(token, buffer, k)
                UpdateTrueCount(trueCount, token)
            
#     print(buffer)
#     nHighestCount(trueCount, 10)
    print(FalsePositiveRate(buffer, trueCount, m, k), '\n')

AmItheAsshole
0.7096774193548387 

AskReddit
0.8108552631578947 

Damnthatsinteresting
0.82174688057041 

DestinyTheGame
0.7767722473604827 

Home
0.8130081300813008 

LivestreamFail
0.6882217090069284 

NoStupidQuestions
0.7209821428571429 

PublicFreakout
0.7712177121771218 

Unexpected
0.8213333333333334 

WhitePeopleTwitter
0.7211981566820277 

antiwork
0.730593607305936 

diablo4
0.7674418604651163 

explainlikeimfive
0.8360655737704918 

facepalm
0.6588235294117647 

funny
0.6917808219178082 

gaming
0.779467680608365 

interestingasfuck
0.7707129094412332 

leagueoflegends
0.8372093023255814 

mildlyinfuriating
0.8069883527454242 

movies
0.7272727272727273 

pcmasterrace
0.7639639639639639 

pics
0.8300907911802854 

therewasanattempt
0.7785714285714286 

videos
0.7808471454880295 

worldnews
0.7780487804878049 



In [33]:
# Prints out the first n tokens with the highest count in a subreddit
def nHighestCount(trueCount, n):
    # Sort the trueCount
    sortedDict = {k: v for k, v in sorted(trueCount.items(), key=lambda item: item[1], reverse=True)}
    
    # Print first n values
    print({k: sortedDict[k] for k in list(sortedDict)[:n]})